## Extraction of Combined Kinematic-Muscular Synergies without Pose Data

This Python script performs a detailed muscle and kinematic synergy analysis on preprocessed experimental data for multiple participants. It is designed to be the final processing step, working from standardized data matrices to produce publication-ready results.

**Key Features:**

* **Works from Correctly Preprocessed Data:** The script loads two separate, pre-standardized data matrices (one for phase1, one for phase2) for each participant.

* **Multi-Level Analysis Loop:** It systematically iterates through Participants, number of Synergies, and Trials.

* **Four Distinct, Separate Analyses per Trial:** For every trial, four analyses are conducted and organized under the `Full Phases` and `Lift-Onset` naming convention:
    * **Full Phases / Phase 1:** Synergies from the complete reach-and-grasp phase.
    * **Full Phases / Phase 2:** Synergies from the complete lift-and-hold phase.
    * **Lift-Onset / Pre-Lift:** Synergies from the last 0.5s of Phase 1.
    * **Lift-Onset / Post-Lift:** Synergies from the first 1.0s of Phase 2.

* **Adaptive MMF Algorithm:** Uses an intelligent Mixed Matrix Factorization (MMF) method with adaptive iterations.

* **Exception Handling:** Handles and flags known missing trials for Participant 7.

* **Organized Output:** All results are saved into a highly organized folder structure.

```markdown
└── P(1)/  (Example for a standard participant)
    └── Synergies Publication/
        ├── Full Phases/
        │   ├── Phase 1/
        │   │   ├── 1_Syn/
        │   │   │   ├── Trial_01/
        │   │   │   │   ├── W_synergies.npy
        │   │   │   │   ├── C_activations.npy
        │   │   │   │   └── synergies_Full_Phases_Phase_1.png
        │   │   │   ├── Trial_02/
        │   │   │   └── ... (up to Trial_24)
        │   │   ├── 2_Syn/
        │   │   └── ... (up to 8_Syn)
        │   │
        │   └── Phase 2/
        │       └── ... (same structure as Phase 1)
        │
        └── Lift-Onset/
            ├── Phase 1/
            │   └── ... (same structure as in Full Phases)
            │
            └── Phase 2/
                └── ... (same structure as in Full Phases)

In [ ]:
import os
import numpy as np
import joblib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import gc

# =====================================================================
# 0) Global Parameters & Configuration
# =====================================================================
BASE_DIR = r"C:\Users\schmi\Documents\Studium\TUM\Masterthesis\Experimental Data"
PARTICIPANTS = list(range(1, 9))
PREPROCESSED_DATA_DIR_NAME = "Preprocessed_Data_Matrix"
N_SYNERGIES_TO_ANALYZE = [1,2,3,4,5]         # For Full-Phase
N_SYNERGIES_LIFT_ONSET = [1,3]         # For Lift-Onset

DTYPE = np.float32
P7_SKIPPED_TRIALS = [20, 22]

# MMF Parameters
MU_S = DTYPE(0.1)
LAMBD_S = DTYPE(50.0)
NUM_EMG_CHANNELS = 117
FIXED_ITERATIONS = 5000
COST_TRACKING_INTERVAL = 100
INIT_SCALE = DTYPE(0.05)

# Data & Clipping Parameters
SAMPLING_RATE = 2000.0
PRE_LIFT_CLIP_SECONDS = 1.0
POST_LIFT_CLIP_SECONDS = 1.5
PRE_LIFT_SAMPLES = int(PRE_LIFT_CLIP_SECONDS * SAMPLING_RATE)
POST_LIFT_SAMPLES = int(POST_LIFT_CLIP_SECONDS * SAMPLING_RATE)


# =====================================================================
# 1) Helper Functions (No changes)
# =====================================================================
def trial_info(trial_number):
    """Returns a dictionary with details for a given trial number."""
    protocol = {1: ("Precision Grasp (Four Fingers and Thumb)", "Precision Handle", 0.25, "Left Lever", "No"), 2: ("Precision Grasp (Four Fingers and Thumb)", "Precision Handle", 0.25, "Left Lever", "Yes"), 3: ("Lateral Pinch Grasp", "Lateral Pinch Handle", 0.25, "Right Lever", "No"), 4: ("Lateral Pinch Grasp", "Lateral Pinch Handle", 0.25, "Right Lever", "Yes"), 5: ("Ball Grasp", "Ball Handle", 0.50, "Left Lever", "No"), 6: ("Ball Grasp", "Ball Handle", 0.50, "Left Lever", "Yes"), 7: ("Precision Grasp (Thumb and Index)", "Precision Handle", 0.25, "Front Lever", "No"), 8: ("Precision Grasp (Thumb and Index)", "Precision Handle", 0.25, "Front Lever", "Yes"), 9: ("Disc Grip", "Disc Handle", 0.50, "Back Lever", "No"), 10: ("Disc Grip", "Disc Handle", 0.50, "Back Lever", "Yes"), 11: ("Power Bar Grasp", "Power Bar Handle", 0.50, "Front Lever", "No"), 12: ("Power Bar Grasp", "Power Bar Handle", 0.50, "Front Lever", "Yes"), 13: ("Precision Grasp (Four Fingers and Thumb)", "Precision Handle", 0.25, "Front Lever", "No"), 14: ("Precision Grasp (Four Fingers and Thumb)", "Precision Handle", 0.25, "Front Lever", "Yes"), 15: ("Lateral Pinch Grasp", "Lateral Pinch Handle", 0.25, "Back Lever", "No"), 16: ("Lateral Pinch Grasp", "Lateral Pinch Handle", 0.25, "Back Lever", "Yes"), 17: ("Ball Grasp", "Ball Handle", 0.50, "Front Lever", "No"), 18: ("Ball Grasp", "Ball Handle", 0.50, "Front Lever", "Yes"), 19: ("Precision Grasp (Thumb and Index)", "Precision Handle", 0.25, "Back Lever", "No"), 20: ("Precision Grasp (Thumb and Index)", "Precision Handle", 0.25, "Back Lever", "Yes"), 21: ("Disc Grip", "Disc Handle", 0.50, "Left Lever", "No"), 22: ("Disc Grip", "Disc Handle", 0.50, "Left Lever", "Yes"), 23: ("Power Bar Grasp", "Power Bar Handle", 0.50, "Right Lever", "No"), 24: ("Power Bar Grasp", "Power Bar Handle", 0.50, "Right Lever", "Yes")}
    if trial_number not in protocol: return None
    tup = protocol[trial_number]
    return {'grasp_type': tup[0], 'handle_type': tup[1], 'weight_kg': tup[2], 'lever_side': tup[3], 'knowledge': tup[4]}

def r2_score_matrix(original_2d: np.ndarray, reconstructed_2d: np.ndarray) -> float:
    """Compute R² for two equally‑shaped matrices using float32 throughout."""
    orig = original_2d.astype(DTYPE, copy=False)
    recon = reconstructed_2d.astype(DTYPE, copy=False)
    if not np.isfinite(recon).all():
        return -np.inf
    diff = np.subtract(orig, recon, dtype=DTYPE)
    ss_res = np.sum(diff * diff, dtype=DTYPE)
    mean_orig = np.mean(orig, dtype=DTYPE, keepdims=True)
    diff_tot = np.subtract(orig, mean_orig, dtype=DTYPE)
    ss_tot = np.sum(diff_tot * diff_tot, dtype=DTYPE)
    if ss_tot == 0: return 1.0
    return float(1.0 - (ss_res / ss_tot))

# =====================================================================
# 2) MMF Core & Cost Function (No changes)
# =====================================================================
def calculate_cost(X, W, C, lambd):
    """Calculates the MMF cost function."""
    reconstruction_error = 0.5 * np.sum((X - W @ C)**2)
    regularization_term = 0.5 * lambd * np.sum(W**2)
    return reconstruction_error + regularization_term

def run_mmf_iterations(X, W, C, k, mu_s, lambd_s, num_iters, track_interval):
    """Run MMF updates with an early exit sanity check."""
    n_vars, n_synergies = W.shape
    normX = np.linalg.norm(X.astype(DTYPE, copy=False))
    if normX == 0: return W, C, []

    mu_W = mu_s / normX
    mu_C = mu_s / normX
    lambd = lambd_s / (n_vars * n_synergies)
    cost_history = []
    cost_threshold = 1.2 * (0.5 * normX**2)

    for i in range(num_iters):
        W_update = mu_W * ((X @ C.T) - (W @ (C @ C.T)) - lambd * W)
        C_update = mu_C * ((W.T @ X) - ((W.T @ W) @ C))
        W += W_update.astype(DTYPE)
        C += C_update.astype(DTYPE)
        W[:k, :] = np.maximum(W[:k, :], 0)
        C = np.maximum(C, 0)

        if i == 200:
            cost = calculate_cost(X, W, C, lambd)
            if not np.isfinite(cost) or cost > cost_threshold:
                return None, None, []

        if i % track_interval == 0:
            cost = calculate_cost(X, W, C, lambd)
            cost_history.append(cost)

    return W, C, cost_history


# =====================================================================
# 3) Plotting Function (No changes)
# =====================================================================
def plot_and_save_synergies(W, C, synergy_out_dir, prefix, index_dict, r2_score):
    """Plots synergies with the specified coloring."""
    n_features, n_synergies = W.shape
    if C.shape[0] != n_synergies: raise ValueError("Number of synergies in W and C must match.")
    otb_start, otb_end = index_dict.get('otb_indices', (0,0))
    myo_start, myo_end = index_dict.get('myo_indices', (0,0))
    kin_hand_start, kin_hand_end = index_dict.get('kin_hand_indices', (0,0))

    colors = ["gray"] * n_features
    for i in range(otb_start, otb_end): colors[i] = '#4F81BD'
    for i in range(myo_start, myo_end): colors[i] = 'pink'
    for i in range(kin_hand_start, kin_hand_end): colors[i] = 'orange'

    os.makedirs(synergy_out_dir, exist_ok=True)
    fig_height = 4 * n_synergies
    fig, axes = plt.subplots(n_synergies, 2, figsize=(20, fig_height), squeeze=False)
    fig.suptitle(f'Synergy Analysis for {prefix} | Overall R²: {r2_score:.4f}', fontsize=16, fontweight='bold')

    for i in range(n_synergies):
        ax_bar = axes[i, 0]
        ax_bar.bar(np.arange(n_features), W[:, i], color=colors, alpha=0.7, width=1.0)
        ax_bar.set_title(f"Synergy {i+1} (W column {i+1})", fontsize=14)
        ax_bar.set_xlabel("Variables", fontsize=12)
        ax_bar.set_ylabel("Synergy Component Value", fontsize=12)
        if otb_end > otb_start: ax_bar.text((otb_start + otb_end)/2, ax_bar.get_ylim()[1]*0.95, 'OTB', ha='center', va='top', fontsize=12, color='#4F81BD')
        if myo_end > myo_start: ax_bar.text((myo_start + myo_end)/2, ax_bar.get_ylim()[1]*0.95, 'Myo', ha='center', va='top', fontsize=12, color='pink')
        if kin_hand_end > kin_hand_start: ax_bar.text((kin_hand_start + kin_hand_end)/2, ax_bar.get_ylim()[1]*0.95, 'Kinematic Hand', ha='center', va='top', fontsize=12, color='orange')
        ax_bar.set_xticks([])
        legend_elems = [Patch(facecolor='#4F81BD', label='OTB'), Patch(facecolor='pink', label='Myo'), Patch(facecolor='orange', label='Kinematic Hand')]
        ax_bar.legend(handles=legend_elems, loc='upper left')
        ax_line = axes[i, 1]
        time_axis = np.arange(C.shape[1]) / SAMPLING_RATE
        ax_line.plot(time_axis, C[i, :], label=f'Activation {i+1}', color='green', linewidth=1.5)
        ax_line.set_title(f"Activation {i+1} (C row {i+1})", fontsize=14)
        ax_line.set_xlabel('Time (s)', fontsize=12)
        ax_line.set_ylabel('Activation', fontsize=12)
        ax_line.grid(True, which='both', linestyle='--', linewidth=0.5)

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    out_filename = f"{prefix}_synergies.png"
    out_path = os.path.join(synergy_out_dir, out_filename)
    plt.savefig(out_path, dpi=150)
    plt.close(fig)

def plot_cost_convergence(cost_history, save_dir, prefix):
    """Plots the convergence of the cost function and saves it."""
    if not cost_history:
        return
    fig, ax = plt.subplots(figsize=(10, 6))
    iterations = np.arange(len(cost_history)) * COST_TRACKING_INTERVAL
    ax.plot(iterations, cost_history, marker='.', linestyle='-', color='b')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Cost Function Value')
    ax.set_title(f'Cost Function Convergence for {prefix}')
    ax.grid(True)
    out_filename = f"{prefix}_cost_convergence.png"
    out_path = os.path.join(save_dir, out_filename)
    plt.savefig(out_path, dpi=150)
    plt.close(fig)

# =====================================================================
# 4) Main Execution Helpers
# =====================================================================

def run_single_mmf(data_matrix: np.ndarray, n_synergies: int, rng: np.random.Generator, W_init=None, C_init=None):
    """Runs MMF, initialized with a provided generator object."""
    if data_matrix is None or data_matrix.shape[0] < 5:
        return None, None, []

    X = data_matrix.T.astype(DTYPE, copy=False)
    n_vars, n_samples = X.shape

    W = rng.random((n_vars, n_synergies), dtype=DTYPE) * INIT_SCALE
    C = rng.random((n_synergies, n_samples), dtype=DTYPE) * INIT_SCALE

    if W_init is not None and C_init is not None and W_init.shape[1] < n_synergies:
        prev_n_syn = W_init.shape[1]
        W[:, :prev_n_syn] = W_init
        C[:prev_n_syn, :] = C_init

    W, C, cost_history = run_mmf_iterations(X, W, C, NUM_EMG_CHANNELS, MU_S, LAMBD_S, FIXED_ITERATIONS, COST_TRACKING_INTERVAL)

    return W, C, cost_history

def run_analysis_block(analysis_path: str, data_matrix: np.ndarray, n_syn: int, trial_idx: int,
                       participant_dir: str, index_dict: dict, is_skipped: bool,
                       master_rng: np.random.Generator, W_init=None, C_init=None): # MODIFIED: Accepts master_rng
    """
    Run and save one full analysis block.
    MODIFIED: Generates and prints a seed for each random repetition.
    """
    analysis_name = os.path.normpath(analysis_path).replace(os.sep, "_")
    save_dir = os.path.join(participant_dir, "Synergies Publication 5X", analysis_path,
                            f"{n_syn}_Syn", f"Trial_{trial_idx:02d}")
    if is_skipped:
        save_dir += "_SKIPPED_TRIAL"
    os.makedirs(save_dir, exist_ok=True)

    if is_skipped or data_matrix is None:
        if is_skipped:
            print("  -> Skipping analysis for this trial as marked.")
        return None, None

    best_r2 = -np.inf
    best_W, best_C, best_cost_history = None, None, []

    if n_syn == 1:
        num_random_reps = 5
        has_warm_start_rep = False
        print(f"  -> Running {analysis_path} for {n_syn} synergy ({num_random_reps} random repetitions)...")
    else:
        num_random_reps = 4
        has_warm_start_rep = True
        print(f"  -> Running {analysis_path} for {n_syn} synergies (1 warm start + {num_random_reps} random repetitions)...")

    total_reps = num_random_reps + (1 if has_warm_start_rep else 0)

    if has_warm_start_rep:
        print(f"    - Repetition 1/{total_reps} (Warm Start with fixed seed: 42)...")
        warm_start_rng = np.random.default_rng(42)
        W, C, cost_history = run_single_mmf(data_matrix, n_syn, warm_start_rng, W_init, C_init)
        if W is not None:
            reconstructed = (W @ C).astype(DTYPE)
            r2 = r2_score_matrix(data_matrix.T.astype(DTYPE, copy=False), reconstructed)
            if r2 > best_r2:
                best_r2, best_W, best_C, best_cost_history = r2, W, C, cost_history
                print(f"      -> R²: {best_r2:.5f} (New Best)")
            else:
                 print(f"      -> R²: {r2:.5f}")
        else:
            print("      -> Run failed (unstable).")

    for i in range(num_random_reps):
        rep_num = i + (2 if has_warm_start_rep else 1)
        # MODIFIED: Generate and print a unique seed for this repetition
        random_seed = master_rng.integers(0, 2**31 - 1)
        print(f"    - Repetition {rep_num}/{total_reps} (Random Start with seed: {random_seed})...")
        
        rep_rng = np.random.default_rng(random_seed)
        W, C, cost_history = run_single_mmf(data_matrix, n_syn, rng=rep_rng)

        if W is not None:
            reconstructed = (W @ C).astype(DTYPE)
            r2 = r2_score_matrix(data_matrix.T.astype(DTYPE, copy=False), reconstructed)
            if r2 > best_r2:
                best_r2, best_W, best_C, best_cost_history = r2, W, C, cost_history
                print(f"      -> R²: {r2:.5f} (New Best)")
            else:
                print(f"      -> R²: {r2:.5f}")
        else:
            print("      -> Run failed (unstable), retrying with new seed.")

    if best_W is None:
        print("  -> MMF failed for all repetitions.")
        return None, None

    print(f"  -> Final best R² for {n_syn} synergies: {best_r2:.5f}")
    np.save(os.path.join(save_dir, "W_synergies.npy"), best_W)
    np.save(os.path.join(save_dir, "C_activations.npy"), best_C)

    plot_prefix = f"synergies_{analysis_name}"
    plot_and_save_synergies(best_W, best_C, save_dir, plot_prefix, index_dict, best_r2)
    plot_cost_convergence(best_cost_history, save_dir, plot_prefix)

    del best_W, best_C, W, C, best_cost_history, cost_history
    gc.collect()

    return np.load(os.path.join(save_dir, "W_synergies.npy")), np.load(os.path.join(save_dir, "C_activations.npy"))

# =====================================================================
# 5) Pipeline Entrypoint
# =====================================================================

def main():
    """
    Main execution function.
    Runs two separate analyses: a clipped "Lift-Onset" analysis and a "Full-Phase" analysis.
    CORRECTED: Ensures that the warm-start for N synergies correctly uses the
    result from the N-1 synergy analysis within the same trial and phase.
    """
    master_rng = np.random.default_rng()

    for pid in PARTICIPANTS:
        participant_str = f"P({pid})"
        participant_dir = os.path.join(BASE_DIR, participant_str)
        pre_dir = os.path.join(participant_dir, PREPROCESSED_DATA_DIR_NAME)

        paths = {
            "p1": os.path.join(pre_dir, f"P{pid}_combined_matrix_phase1.npy"),
            "p2": os.path.join(pre_dir, f"P{pid}_combined_matrix_phase2.npy"),
            "idx": os.path.join(pre_dir, f"P{pid}_feature_indices.joblib")
        }
        if not all(os.path.exists(p) for p in paths.values()):
            print(f"ERROR: Preprocessed files not found for P{pid}. Skipping participant.")
            continue

        print("\n" + "=" * 80 + f"\nProcessing Participant: {pid}\n" + "=" * 80)
        
        p1_full_mmap = np.load(paths["p1"], mmap_mode='r')
        p2_full_mmap = np.load(paths["p2"], mmap_mode='r')
        index_dict = joblib.load(paths["idx"])

        if "phase1_trial_lengths" not in index_dict or "phase2_trial_lengths" not in index_dict:
            print(f"ERROR: Trial length information missing; update preprocessing script.")
            continue

        p1_lens, p2_lens = index_dict["phase1_trial_lengths"], index_dict["phase2_trial_lengths"]
        skip_list = P7_SKIPPED_TRIALS if pid == 7 else []
        valid_trials = [t for t in range(1, 25) if t not in skip_list]

        if len(p1_lens) != len(valid_trials) or len(p2_lens) != len(valid_trials):
            print(f"ERROR: Length mismatch for P{pid} between trials and recorded lengths. Skipping.")
            continue

        p1_map, p2_map = {}, {}
        p1_row, p2_row = 0, 0
        for trial_idx_map, p1_len, p2_len in zip(valid_trials, p1_lens, p2_lens):
            p1_map[trial_idx_map] = (p1_row, p1_row + p1_len)
            p2_map[trial_idx_map] = (p2_row, p2_row + p2_len)
            p1_row += p1_len
            p2_row += p2_len

        for trial_idx in range(1, 25):
            print(f"\n{'*' * 20} Processing Trial {trial_idx} {'*' * 20}")
            is_skip = (pid == 7 and trial_idx in skip_list)
            
            # --- Extract data for BOTH analysis types ---
            lift_p1, lift_p2, full_p1, full_p2 = None, None, None, None
            if not is_skip and trial_idx in p1_map and trial_idx in p2_map:
                p1_sl = slice(*p1_map[trial_idx])
                p2_sl = slice(*p2_map[trial_idx])
                
                # Data for Lift-Onset analysis (clipped)
                lift_p1 = p1_full_mmap[p1_sl][-PRE_LIFT_SAMPLES:].astype(DTYPE)
                lift_p2 = p2_full_mmap[p2_sl][:POST_LIFT_SAMPLES].astype(DTYPE)

                # Data for Full-Phase analysis (un-clipped)
                full_p1 = p1_full_mmap[p1_sl].astype(DTYPE)
                full_p2 = p2_full_mmap[p2_sl].astype(DTYPE)
            else:
                is_skip = True

            # --- 1. LIFT-ONSET ANALYSIS ---
            print("\n--- Running Lift-Onset Analysis (Synergies: 1-3) ---")
            # Initialize warm-start matrices for this trial's Lift-Onset analysis
            prev_W_lift_p1, prev_C_lift_p1 = None, None
            prev_W_lift_p2, prev_C_lift_p2 = None, None
            for n_syn in N_SYNERGIES_LIFT_ONSET:
                print(f"\n-- P{pid}, N_Syn={n_syn}, Trial={trial_idx} --")
                
                path1 = os.path.join("Lift-Onset", "Phase 1")
                W_res_1, C_res_1 = run_analysis_block(path1, lift_p1, n_syn, trial_idx, participant_dir, index_dict, is_skip, master_rng, W_init=prev_W_lift_p1, C_init=prev_C_lift_p1)
                if W_res_1 is not None:
                    prev_W_lift_p1, prev_C_lift_p1 = W_res_1, C_res_1
                
                path2 = os.path.join("Lift-Onset", "Phase 2")
                W_res_2, C_res_2 = run_analysis_block(path2, lift_p2, n_syn, trial_idx, participant_dir, index_dict, is_skip, master_rng, W_init=prev_W_lift_p2, C_init=prev_C_lift_p2)
                if W_res_2 is not None:
                    prev_W_lift_p2, prev_C_lift_p2 = W_res_2, C_res_2

            # --- 2. FULL-PHASE ANALYSIS ---
            print("\n--- Running Full-Phase Analysis (Synergies: 1-7) ---")
            # Initialize warm-start matrices for this trial's Full-Phase analysis
            prev_W_full_p1, prev_C_full_p1 = None, None
            prev_W_full_p2, prev_C_full_p2 = None, None
            for n_syn in N_SYNERGIES_TO_ANALYZE:
                print(f"\n-- P{pid}, N_Syn={n_syn}, Trial={trial_idx} --")

                path1 = os.path.join("Full-Phase", "Phase 1")
                W_res_1, C_res_1 = run_analysis_block(path1, full_p1, n_syn, trial_idx, participant_dir, index_dict, is_skip, master_rng, W_init=prev_W_full_p1, C_init=prev_C_full_p1)
                if W_res_1 is not None:
                    prev_W_full_p1, prev_C_full_p1 = W_res_1, C_res_1

                path2 = os.path.join("Full-Phase", "Phase 2")
                W_res_2, C_res_2 = run_analysis_block(path2, full_p2, n_syn, trial_idx, participant_dir, index_dict, is_skip, master_rng, W_init=prev_W_full_p2, C_init=prev_C_full_p2)
                if W_res_2 is not None:
                    prev_W_full_p2, prev_C_full_p2 = W_res_2, C_res_2

    print("\nSelective regeneration complete for all specified participants and synergy levels.")

if __name__ == "__main__":
    main()


Processing Participant: 1

******************** Processing Trial 1 ********************

--- Running Lift-Onset Analysis (Synergies: 1-3) ---

-- P1, N_Syn=1, Trial=1 --
  -> Running Lift-Onset\Phase 1 for 1 synergy (5 random repetitions)...
    - Repetition 1/5 (Random Start with seed: 1248724113)...
      -> R²: 0.96081 (New Best)
    - Repetition 2/5 (Random Start with seed: 1994742490)...
      -> R²: 0.96081
    - Repetition 3/5 (Random Start with seed: 2052793459)...
      -> R²: 0.96081
    - Repetition 4/5 (Random Start with seed: 1053454869)...
      -> R²: 0.96081
    - Repetition 5/5 (Random Start with seed: 458307680)...
      -> R²: 0.96081
  -> Final best R² for 1 synergies: 0.96081
  -> Running Lift-Onset\Phase 2 for 1 synergy (5 random repetitions)...
    - Repetition 1/5 (Random Start with seed: 580736302)...
      -> R²: 0.99670 (New Best)
    - Repetition 2/5 (Random Start with seed: 966756322)...
      -> R²: 0.99670
    - Repetition 3/5 (Random Start with seed: 17